### TripAdvisor Scraper

#### Import and Load

In [1]:
import pandas as pd
from datetime import datetime, timedelta
datetimify_timestamp = lambda s: datetime.strptime(s, '%Y%m%d')

from bs4 import BeautifulSoup
import requests

# Load data
df_train = pd.read_csv('main_task.csv')
df_test = pd.read_csv('kaggle_task.csv')

#### Create Dataframe

In [211]:
df_test.drop('Name', axis=1, inplace=True)

df = df_test.append(df_train, sort=False).reset_index(drop=True)

In [218]:
url_ta_column = 'URL_TA'
id_ta_column = 'ID_TA'
prices_column = 'Price Range'
reviews_column = 'Number of Reviews'
cuisines_column = 'Cuisine Style'

nan_prices = df[df[prices_column].isna()][id_ta_column]
nan_reviews = df[df[reviews_column].isna()][id_ta_column]
nan_cuisines = df[df[cuisines_column].isna()][id_ta_column]

all_id_tas = df[id_ta_column]

#### Configure Parsing Settings

In [224]:
class__prices_range = 'prw_rup prw_restaurants_restaurant_detail_tags tagsContainer'
class__review_count = 'reviewCount'
class__cuisines = 'restaurants-detail-overview-cards-DetailsSectionOverviewCard__tagText--1OH6h'
class__ratings = 'restaurants-detail-overview-cards-RatingsOverviewCard__ratingQuestionRow--5nPGK'


base_url = 'https://www.tripadvisor.com'

In [10]:
prices_range = page.find('div', class_=class__prices_range).find('a').text
review_count = page.find('span', class_=class__review_count).text
cuisines = page.find('div', text='CUISINES').findNextSibling().text

print(prices_range)
print(review_count)
print(cuisines)
print(response.url)

$$$$
265 reviews
Italian, Mediterranean, European
https://www.tripadvisor.com/Restaurant_Review-g186338-d680417-Reviews-Quirinale-London_England.html


In [210]:
# Check whether any rest pages URLs changed
def find_diff(url1, url2):
    old = url1.split('-')
    new = url2.split('-')
    
    changes = ''
    for i in range(len(old)):
        if old[i] != new[i]:
            changes += '[' + old[i] + ' --> ' + new[i] + '] '
    return changes

for i in range(10):
    add_url = df.iloc[i][url_ta_column]
    url = base_url + add_url
    response = requests.get(url)
    page = BeautifulSoup(response.text, 'html.parser')
    if response.url != url:
        print(i, ':', find_diff(url, response.url))

0 : [d10746918 --> d4990732] 
1 : [Ravintola_Kolmon3n --> Restaurant_Kolmon3n] 


In [273]:
def fetch_page(i):
    add_url = df[df[id_ta_column] == i].iloc[0][url_ta_column]
    r = requests.get(base_url + add_url)
    return BeautifulSoup(r.text, 'html.parser')

def get_prices(i):
    f1 = fetch_page(i).find('div', class_ = class__prices_range)
    if f1:
        f2 = f1.find('a')
        pr = f2.text if f2 else ''
        if '$' in pr: return [i, pr]
    return None

def get_reviews(i):
    f1 = fetch_page(i).find('span', class_ = class__review_count)
    if f1:
        pr = f1.text.replace(' review', '').replace('s', '').replace(',', '')
        return [i, int(pr)]
    return None

def get_ratings(i):
    ratings = []
    f1 = fetch_page(i).find_all(class_ = 'ui_bubble_rating')[2:6]
    for r in f1:
        ratings.append(float(r.attrs['class'][1][-2:]) / 10)
    return ratings

In [282]:
ratings_columns = ['rating_Food', 'rating_Service', 'rating_Value', 'rating_Atmosphere']

1.6 s ± 209 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [69]:
from multiprocessing.dummy import Pool as ThreadPool
pool = ThreadPool()

results_prices = pool.map(get_prices, nan_prices[:100])

pool.close()
pool.join()

In [72]:
results_prices

[['d8286886', '$'],
 ['d12947542', '$$ - $$$'],
 ['d12716384', '$$ - $$$'],
 ['d7818546', '$$ - $$$'],
 ['d1344523', '$$ - $$$'],
 None,
 None,
 None,
 ['d5795143', '$$ - $$$'],
 None,
 ['d13234520', '$$ - $$$'],
 None,
 ['d8180306', '$'],
 None,
 None,
 None,
 ['d9561771', '$$ - $$$'],
 None,
 ['d11633790', '$'],
 None,
 ['d8448700', '$'],
 ['d5776216', '$$ - $$$'],
 None,
 None,
 ['d8338351', '$'],
 ['d8352929', '$$ - $$$'],
 ['d719850', '$$ - $$$'],
 None,
 ['d6962328', '$'],
 None,
 ['d8837875', '$'],
 ['d7813343', '$$ - $$$'],
 None,
 ['d9762124', '$$ - $$$'],
 ['d8376187', '$$ - $$$'],
 None,
 ['d9559203', '$$ - $$$'],
 None,
 ['d10455057', '$'],
 None,
 None,
 None,
 None,
 ['d7818336', '$$ - $$$'],
 None,
 None,
 ['d12286606', '$'],
 ['d2284554', '$$ - $$$'],
 None,
 None,
 ['d12172104', '$'],
 None,
 ['d12673809', '$$ - $$$'],
 ['d12662600', '$$ - $$$'],
 ['d5820527', '$'],
 ['d5796259', '$'],
 ['d6016792', '$'],
 ['d8580647', '$'],
 ['d5223231', '$'],
 ['d2244203', '$$ - $$$'

In [73]:
pool = ThreadPool()

results_reviews = pool.map(get_reviews, nan_reviews[:1000])

pool.close()
pool.join()

In [75]:
len(results_reviews)

1000

In [44]:
result_reviews = list(filter(None, result_reviews))
print(len(result_reviews))

3200

In [5]:
df_result_reviews = pd.DataFrame(result_reviews,
                                 columns=[id_ta_column, reviews_column])


print(df_result_reviews)

NameError: name 'result_reviews' is not defined

In [ ]:
df_result_reviews.to_csv('parsed_reviews.csv', index=False)

In [4]:
df_results_loaded = pd.read_csv('parsed_reviews.csv')

df_results_loaded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3157 entries, 0 to 3156
Data columns (total 2 columns):
ID_TA                3157 non-null object
Number of Reviews    3157 non-null int64
dtypes: int64(1), object(1)
memory usage: 49.5+ KB


In [95]:
webarchive_availability_url = 'http://archive.org/wayback/available?url='

def fetch_webarchive(i):
    add_url = df[df['ID_TA'] == i].iloc[0]['URL_TA']
    r = requests.get(webarchive_availability_url + base_url + add_url + '&timestamp=2017')
    return r.json()

In [183]:
def parse_webarchive_page(id_ta):
    page = fetch_webarchive(id_ta)
    info = page['archived_snapshots']
    if info != {}:
        return {id_ta: info['closest']['timestamp']}

In [184]:
# TripAdvisor server doesn't withstand a load of more than 2 Threads at once
pool = ThreadPool(2)

archived_pages = pool.map(parse_webarchive_page, nan_reviews[:100])

pool.close()
pool.join()

In [185]:
archived_pages

[{'d8286886': '20151202020851'},
 {'d13202032': '20190823021814'},
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 {'d753088': '20151201203102'},
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 {'d7287872': '20190814182237'},
 {'d8364955': '20190928215018'},
 None,
 {'d7123259': '20191004040557'},
 None,
 None,
 None,
 None,
 None,
 None,
 {'d12151343': '20190924005136'},
 None,
 {'d12145892': '20190917081708'},
 None,
 None,
 None,
 None,
 {'d7701632': '20190909210139'},
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 {'d10054994': '20190914172637'},
 None,
 None,
 {'d7124372': '20191001201534'},
 None,
 None,
 None,
 {'d7234492': '20191101080210'},
 None,
 None,
 None,
 {'d10521562': '20190519080408'},
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 {'d815943': '20191023104409'},
 {'d5872661': '20151231230440'},
 None,
 None,
 {'d11737024': '20190910004132'},
 None,
 {'d986295': '20191018092726'},
 None,
 None,
 N